In [ ]:
#TITANIC SURVIVOR ANALYSIS
import pandas as pd
import numpy as np


In [ ]:
#Use of pandas read_csv function to read the csv file in python.
#Use of pandas DataFrame method to convert the file into data frame.
df=pd.DataFrame(pd.read_csv('/content/train (1).csv'))
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [ ]:
df.shape

(891, 12)

HANDELING OF NULL VALUES:

The dataset may contain rows or columns for which several values will be missing. We have to judge that whether to drop the entire row or column or fill those spaces with some suitable values based on how significant the row or column is for our analysis.

In [ ]:
# Obtaining the number of null values present in each column.
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

Here we observe that the columns corresponding to age, cabin and embarked have null values. Out of these the cabin column corresponds to maximum number of null values.

In [ ]:
#Separating out the columns which have more than 35% of the data as null
x = df.isnull().sum()
drop_col= x[x>(35/100 * df.shape[0])]
drop_col

Cabin    687
dtype: int64

In [ ]:
drop_col.index

Index(['Cabin'], dtype='object')

In [ ]:
df.drop(drop_col.index, axis=1, inplace=True)
df.isnull().sum()


PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Embarked         2
dtype: int64

After removing the cabin column we now need to deal with the null values of age and embarked column.

In [ ]:
# Filling the null values of the age with the mean value of the age values available.
df.fillna(df.mean(),inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       2
dtype: int64

In [ ]:
#Filling the embarked null values with that value of embarked which has highest frequency
df['Embarked'].describe()

count     889
unique      3
top         S
freq      644
Name: Embarked, dtype: object

Here we see that the value 'S'is occuring maximum number of times in the Embarked column. So, we fill the null values of Embarked with 'S' as it was not possible to find the mean value of the data present in Embarked.

In [ ]:
df['Embarked'].fillna('S', inplace=True)
df.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Embarked       0
dtype: int64

In [ ]:
# observing the correlation values for different parameters
df.corr()

,PassengerId,Survived,Pclass,Age,SibSp,Parch,Fare
PassengerId,1.000000,-0.005007,-0.035144,0.033207,-0.057527,-0.001652,0.012658
Survived,-0.005007,1.000000,-0.338481,-0.069809,-0.035322,0.081629,0.257307
Pclass,-0.035144,-0.338481,1.000000,-0.331339,0.083081,0.018443,-0.549500
Age,0.033207,-0.069809,-0.331339,1.000000,-0.232625,-0.179191,0.091566
SibSp,-0.057527,-0.035322,0.083081,-0.232625,1.000000,0.414838,0.159651
Parch,-0.001652,0.081629,0.018443,-0.179191,0.414838,1.000000,0.216225
Fare,0.012658,0.257307,-0.549500,0.091566,0.159651,0.216225,1.000000


Here we can obseve that:

1) There is a significant correlation between the survived parameter and    pclass parameter in negative sense. Also pclass has good correlation between with age also in negative sense.

2) Pclass has a great correlation value (more significant than survived and age parameters) with fare in negative sense.

3) Parch and sibsp share a good correlation in positive sense.

In [ ]:
# Combing Parch and Sibsp into single column as both of them represent family.
# And both have good correlation value in positive sense
df['Familysize']= df['SibSp'] + df ['Parch']
df.drop(['SibSp','Parch'], axis=1, inplace=True)
df.corr()

,PassengerId,Survived,Pclass,Age,Fare,Familysize
PassengerId,1.000000,-0.005007,-0.035144,0.033207,0.012658,-0.040143
Survived,-0.005007,1.000000,-0.338481,-0.069809,0.257307,0.016639
Pclass,-0.035144,-0.338481,1.000000,-0.331339,-0.549500,0.065997
Age,0.033207,-0.069809,-0.331339,1.000000,0.091566,-0.248512
Fare,0.012658,0.257307,-0.549500,0.091566,1.000000,0.217138
Familysize,-0.040143,0.016639,0.065997,-0.248512,0.217138,1.000000


Familysize do not have much correlation with survived.

In [ ]:
# categorizing alone or not based on the familysize to see whether alone has good correlation with surivival rate or not.
df['Alone']=[0 if df['Familysize'][i]>0 else 1 for i in df.index]
df.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,Ticket,Fare,Embarked,Familysize,Alone
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,A/5 21171,7.2500,S,1,0
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,PC 17599,71.2833,C,1,0
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,STON/O2. 3101282,7.9250,S,0,1
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,113803,53.1000,S,1,0
4,5,0,3,"Allen, Mr. William Henry",male,35.0,373450,8.0500,S,0,1


Value 0 means the person is not Alone.

Value 1 means the person is Alone.

In [ ]:
df.groupby(['Alone'])['Survived'].mean()

Alone
0    0.505650
1    0.303538
Name: Survived, dtype: float64

Here we can observe that person who is not alone has a greater chance of survival.

As rich people who are might be travelling as passengers along with their families were priortized over alone travelling crew members.

In [ ]:
#Correlating the fare with alone parameter.
df[['Alone','Fare']].corr()

,Alone,Fare
Alone,1.000000,-0.271832
Fare,-0.271832,1.000000


Here we observe that if the person is not alone the ticket price is higher than the case when the person is alone. 

This supports our previous conclusion drawn that rich people are travelling as passengers along with their families.

In [ ]:
# 1 for female and 0 for male
df['Sex']= [0 if df['Sex'][i]=='male' else 1 for i in df.index]
df.groupby(['Sex'])['Survived'].mean()

Sex
0    0.188908
1    0.742038
Name: Survived, dtype: float64

IT shows that female passengers have more chance of survival that means the female passengers were priortized over male.

In [ ]:
df.groupby(['Embarked'])['Survived'].mean()

Embarked
C    0.553571
Q    0.389610
S    0.339009
Name: Survived, dtype: float64

CONCLUSIONS:

1) Female passengers were more likely to survive.

2) People with families were mostly rich people and were priortized over alone passengers.

3) People Embarked from port C had more survival chances and from port S least.